In [ ]:
# MNIST example : https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist
# Taxi fare example : https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb#scrollTo=4zyIqWl9TSdG
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10  -> use in terminal

In [1]:
import logging
import os

import absl
import tensorflow_model_analysis as tfma
from typing import Any, Dict, Iterable, List, Text
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components import ImportExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, ResolverNode, Evaluator, Pusher
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata, pipeline
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.utils.dsl_utils import external_input
!pip install git+https://github.com/JanMarcelKezmann/TensorFlow-Advanced-Segmentation-Models.git

/home/vandan/miniconda3/envs/CV/lib/python3.6/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


  Cloning https://github.com/JanMarcelKezmann/TensorFlow-Advanced-Segmentation-Models.git to /tmp/pip-req-build-_amhfmgy
  Running command git clone -q https://github.com/JanMarcelKezmann/TensorFlow-Advanced-Segmentation-Models.git /tmp/pip-req-build-_amhfmgy


In [2]:
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [41]:

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_name = "TFx_seg"
_pipeline_root = "./"

# This is the path where your model will be pushed for serving.
_serving_model_dir = "./"
# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import \
InteractiveContext
context = InteractiveContext()

In [5]:
op = example_gen_pb2.Input(
        splits=[
          example_gen_pb2.Input.Split(name='train', pattern='./data/tfrecords/train/*'),
          example_gen_pb2.Input.Split(name="eval", pattern="./data/tfrecords/test/*")
          ]
        )
base_dir = ""
examples = external_input(base_dir)
example_gen = ImportExampleGen(input=examples, input_config=op)
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


INFO:absl:Running driver for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for ImportExampleGen
INFO:absl:Generating examples.


INFO:absl:Reading input TFRecord data ./data/tfrecords/train/*.
INFO:absl:Reading input TFRecord data ./data/tfrecords/test/*.
INFO:absl:Examples generated.
INFO:absl:Running publisher for ImportExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:2236502,xor_checksum:1612714958,sum_checksum:1612714958\nsplit:eval,num_files:1,total_bytes:2800359,xor_checksum:1612714958,sum_checksum:1612714958"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [6]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/ImportExampleGen/examples/1


In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

In [7]:
from tfx.components import StatisticsGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [8]:
import tensorflow_data_validation as tfdv
stats = tfdv.generate_statistics_from_tfrecord(data_location='./data/tfrecords/train/train.tfrecords')
stats

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


datasets {
  num_examples: 10
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 10
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            

In [9]:
import tensorflow_data_validation as tfdv
stats = tfdv.generate_statistics_from_tfrecord(data_location='./data/tfrecords/test/test.tfrecords')
stats

datasets {
  num_examples: 10
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 10
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1.0
          }
          buckets {
            low_value: 1.0
            

In [10]:
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'image_raw',BYTES,required,,-
'mask_raw',BYTES,required,,-


In [11]:
from tfx.components import SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=True)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [ ]:
context.show(schema_gen.outputs['schema'])

In [12]:
from tfx.components import ExampleValidator
example_validator = ExampleValidator(
statistics=statistics_gen.outputs['statistics'],
schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/ExampleValidator/anomalies/4/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/ExampleValidator/anomalies/4/eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [ ]:
context.show(example_validator.outputs['anomalies'])

In [26]:
_constants_module_file = 'constants.py'

In [27]:
%%writefile {_constants_module_file}
IMAGE_KEY = "image_raw"
MASK_KEY = "mask_raw"

EPOCHS = 1
TRAIN_STEPS = 1
EVAL_STEPS = 2
TRAIN_BATCH_SIZE = 3
EVAL_BATCH_SIZE = 3
HEIGHT = 320
WIDTH = 320
PRETRAINED_WEIGHTS = "imagenet"
BACKBONE_TRAINABLE = False
BACKBONE_NAME = "efficientnetb3"

TOTAL_CLASSES = ['sky', 'building', 'pole', 'road', 'pavement', 
               'tree', 'signsymbol', 'fence', 'car', 
               'pedestrian', 'bicyclist', 'unlabelled']
N_TOTAL_CLASSES = 12
MODEL_CLASSES = ["sky", "building"]
if len(MODEL_CLASSES) == 11:
    ALL_CLASSES = True
    MODEL_CLASSES = TOTAL_CLASSES
    N_MODEL_CLASSES = N_TOTAL_CLASSES
else:
    ALL_CLASSES = False
    N_MODEL_CLASSES = len(MODEL_CLASSES) + 1

Overwriting constants.py


In [17]:
_transform_module_file = 'transform.py'

In [18]:
%%writefile {_transform_module_file}

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from typing import Union, List, Text
import random

import numpy as np
import albumentations as A
import tensorflow.keras.backend as K

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_advanced_segmentation_models as tasm

import constants


def _transformed_name(key: Text) -> Text:
    """Generate the name of the transformed feature from original name."""
    return key + "_xf"


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _get_serve_image_fn(model):
  """Returns a function that feeds the input tensor into the model."""

  @tf.function
  def serve_image_fn(image_tensor):
    """Returns the output to be used in the serving signature.
    
    Args:
        image_tensor: A tensor represeting input image. The image should have 3
                      channels.
    
    Returns:
        The model's predicton on input image tensor
    """
    return model(image_tensor)

  return serve_image_fn

# TFx transform comp will call this func
def preprocessing_fn(inputs: tf.Tensor) -> tf.Tensor:
    """tf.transform's callback function for preprocessing inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      outputs: map from feature keys to raw not-yet-transformed features
    """
    outputs = {}
    
    image_features = tf.map_fn(
        lambda x: tf.io.decode_png(x[0], channels=3),
        inputs[constants.IMAGE_KEY],
        dtype=tf.uint8
        )
    
    image_features = tf.image.resize(image_features, [constants.HEIGHT, constants.WIDTH])
    image_features = tf.image.per_image_standardization(image_features)

    mask_features = tf.map_fn(
        lambda x: tf.io.decode_png(x[0], channels=3),
        inputs[constants.MASK_KEY],
        dtype=tf.uint8
        )

    mask_features = tf.image.resize(mask_features, [constants.HEIGHT, constants.WIDTH])
    mask_features = tf.math.reduce_max(mask_features, axis=-1)
    mask_features = tf.cast(mask_features, dtype=tf.int32)
    mask_features = tf.one_hot(indices=mask_features, depth=constants.N_TOTAL_CLASSES)
    class_values = [constants.TOTAL_CLASSES.index(cls.lower()) for cls in constants.MODEL_CLASSES]
    if not constants.ALL_CLASSES:
        fg_list = []
        bg_list = []
        for mask_num in range(constants.N_TOTAL_CLASSES):
            if mask_num in class_values:
                # Add mask of a class to new_mask
                fg_list.append(mask_features[:, :, :, mask_num])
            else:
                # add all class masks belonging to the background to the background class of the new_mask
                bg_list.append(mask_features[:, :, :, mask_num])

        bg = tf.math.reduce_sum(tf.stack(bg_list, axis=-1), axis=-1, keepdims=False)
        fg_list.append(bg)
        mask_features = tf.stack(fg_list, axis=-1)

    outputs[_transformed_name(constants.IMAGE_KEY)] = image_features
    outputs[_transformed_name(constants.MASK_KEY)] = mask_features
    return outputs


Overwriting transform.py


In [19]:
transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(_transform_module_file)
        )
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.


Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
Use ref() instead.


INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:We decided to produce LargeList and LargeBinary types.
INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:We decided to produce Lar

...

INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.


INFO:absl:Feature image_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature mask_raw has a shape dim {
  size: 1
}
. Setting to DenseTensor.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 14
        name: "TransformCache"
        )]
        ))

In [20]:
_model_module_file = 'model.py'

In [21]:
%%writefile {_model_module_file}

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from typing import Union, List, Text
import random

import numpy as np
import albumentations as A
import tensorflow.keras.backend as K

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_advanced_segmentation_models as tasm

import constants


def _transformed_name(key: Text) -> Text:
    """Generate the name of the transformed feature from original name."""
    return key + "_xf"


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _get_serve_image_fn(model):
  """Returns a function that feeds the input tensor into the model."""

  @tf.function
  def serve_image_fn(image_tensor):
    """Returns the output to be used in the serving signature.
    
    Args:
        image_tensor: A tensor represeting input image. The image should have 3
                      channels.
    
    Returns:
        The model's predicton on input image tensor
    """
    return model(image_tensor)

  return serve_image_fn


def _image_and_mask_augmentation(image_features, mask_features):
    """Perform image augmentation on batches of images .
    
    Args:
        image_features: a batch of image features
        mask_features: a batch of mask features
    
    Returns:
        The augmented image and mask features
    """
    batch_size = tf.shape(image_features)[0]
    seed = random.random()
    image_features = tf.image.random_flip_left_right(image_features, seed=seed)
    mask_features = tf.image.random_flip_left_right(mask_features, seed=seed)
    image_features = tf.image.resize_with_crop_or_pad(image_features, constants.HEIGHT + 30, constants.WIDTH + 30)
    mask_features = tf.image.resize_with_crop_or_pad(mask_features, constants.HEIGHT + 30, constants.WIDTH + 30)
    image_features = tf.image.random_crop(image_features, (batch_size, constants.HEIGHT, constants.WIDTH, 3), seed=seed)
    mask_features = tf.image.random_crop(mask_features, (batch_size, constants.HEIGHT, constants.WIDTH, len(constants.MODEL_CLASSES) + 1), seed=seed)
    return image_features, mask_features


def _data_augmentation(feature_dict, mask_features):
    """Perform data augmentation on batches of data.
    
    Args:
        feature_dict: a dict containing features of samples
    
    Returns:
        The feature dict with augmented features
    """
    image_features = feature_dict[_transformed_name(constants.IMAGE_KEY)]
    image_features, mask_features = _image_and_mask_augmentation(image_features, mask_features)
    feature_dict[_transformed_name(constants.IMAGE_KEY)] = image_features
    return feature_dict, mask_features


def _input_fn(file_pattern: List[Text], 
              tf_transform_output: tft.TFTransformOutput, 
              batch_size=4,
              is_train: bool=False) -> tf.data.Dataset:
    """Generates features and label for tuning/training.
    
    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of returned
                    dataset to combine in a single batch
        is_train: Whether the input dataset is train split or not

    Returns:
        A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
        )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=_transformed_name(constants.MASK_KEY),
        prefetch_buffer_size=2,
        num_epochs=constants.EPOCHS
        )

    if is_train:
        dataset = dataset.map(lambda x, y: (_data_augmentation(x, y)))

    return dataset

# model 
def get_model(fn_args):
    """
    This function defines a Keras model and returns the model as a Keras object.
    """
    base_model, layers, layer_names = tasm.create_base_model(
        name=constants.BACKBONE_NAME,
        weights=constants.PRETRAINED_WEIGHTS,
        height=constants.HEIGHT,
        width=constants.WIDTH,
        include_top=False,
        pooling=None
    )

    model = tasm.DANet(
        n_classes=constants.N_MODEL_CLASSES,
        base_model=base_model,
        output_layers=layers,
        backbone_trainable=constants.BACKBONE_TRAINABLE,
        height=constants.HEIGHT,
        width=constants.WIDTH
    ).model()

    opt = tf.keras.optimizers.SGD(learning_rate=0.2, momentum=0.9)
    metrics = [tasm.metrics.IOUScore(threshold=0.5)]
    categorical_focal_dice_loss = tasm.losses.CategoricalFocalLoss(alpha=0.25, gamma=2.0) + tasm.losses.DiceLoss()

    model.compile(
        optimizer=opt,
        loss=categorical_focal_dice_loss,
        metrics=metrics,
    )

    return model


# TFX Trainer will call this function.
def run_fn(fn_args):
    """Train the model based on given args.

    Args:
        fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    TrainSetwoAug = _input_fn(
        fn_args.train_files,
        tf_transform_output,
        constants.TRAIN_BATCH_SIZE,
        is_train=True
    )

    ValidationSet = _input_fn(
        fn_args.eval_files,
        tf_transform_output,
        constants.EVAL_BATCH_SIZE,
        is_train=False
    )

    model = get_model(fn_args)

    try:
        log_dir = fn_args.model_run_dir
    except KeyError:
        log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    callbacks = [
                 tf.keras.callbacks.ReduceLROnPlateau(monitor="iou_score", factor=0.2, patience=6, verbose=1, mode="max"),
                 tf.keras.callbacks.EarlyStopping(monitor="iou_score", patience=16, mode="max", verbose=1, restore_best_weights=True),
                 tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq="batch")
    ]

    print("Start Training")

    model.fit(
        TrainSetwoAug,
        epochs=constants.EPOCHS,
        steps_per_epoch=constants.TRAIN_STEPS,
        validation_data=ValidationSet,
        validation_steps=constants.EVAL_STEPS,
        callbacks=callbacks,
    )

    print("Training Finished")

    signatures = {
        'serving_default':
            _get_serve_image_fn(model).get_concrete_function(
                tf.TensorSpec(
                    shape=[None, constants.HEIGHT, constants.WIDTH, 3],
                    dtype=tf.float32,
                    name=_transformed_name(constants.IMAGE_KEY)
                )
            )
    }

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

    print("Model Saved")

Overwriting model.py


In [22]:
trainer = Trainer(
        module_file=os.path.abspath(_model_module_file),
        custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(splits=["train"]),
        eval_args=trainer_pb2.EvalArgs(splits=["eval"]),
        )
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.


Start Training
1/1 [==============================] - ETA: 0s - loss: 0.7657 - iou_score: 0.084 - 25s 25s/step - loss: 0.7657 - iou_score: 0.0842 - val_loss: 0.7347 - val_iou_score: 0.1226
Training Finished
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Trainer/model/6/serving_model_dir/assets
Model Saved


INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Trainer/model/6/serving_model_dir. ModelRun written to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Trainer/model_run/6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 16
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 17
        name: "ModelRun"
        )]
        ))

In [24]:
model_resolver = ResolverNode(
        instance_name='latest_blessed_model_resolver',
        resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
        )
context.run(model_resolver)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [47]:
serving_model_dir = './'
pusher = Pusher(
        model=trainer.outputs['model'],
        # model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir
                )
            )
        )
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1612955699
INFO:absl:Model written to serving path ./1612955699.
INFO:absl:Model pushed to /tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Pusher/pushed_model/8.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 8
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 10
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-02-10T15_59_52.740346-d5fzxz24/Pusher/pushed_model/8"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "./1612955699"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1612955699"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 20
        name: "PushedModel"
        )]
        ))

In [ ]:
push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)